In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df_principal = pd.read_excel('acoes-perso.xlsx', sheet_name='Principal')
df_principal

In [ ]:
df_total_acoes = pd.read_excel('acoes-perso.xlsx', sheet_name='Total_de_acoes')
df_total_acoes

In [ ]:
df_tickers = pd.read_excel('acoes-perso.xlsx', sheet_name='Ticker')
df_tickers

In [ ]:
df_chatgpt = pd.read_excel('acoes-perso.xlsx', sheet_name='ChatGPT')
df_chatgpt

In [ ]:
df_principal = df_principal[['Ativo', 'Data', 'Último (R$)', 'Var. Dia (%)']].copy()
df_principal

In [ ]:
df_principal = df_principal.rename(columns={'Último (R$)':'valor_final', 'Var. Dia (%)':'var_dia_pct'}).copy()
df_principal

In [ ]:
df_principal['var_pct'] = df_principal['var_dia_pct'] / 100
df_principal['valor_inicial'] = df_principal['valor_final'] / (df_principal['var_pct'] + 1)
df_principal

In [ ]:
df_principal = df_principal.merge(df_total_acoes, left_on='Ativo', right_on='Código', how='left')
df_principal

In [ ]:
df_principal = df_principal.drop(columns=['Código'])
df_principal

In [ ]:
df_principal['Variação_rs'] = (df_principal['valor_final'] - df_principal['valor_inicial']) * df_principal['Qtde. Teórica']
df_principal

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
df_principal

In [ ]:
df_principal['Qtde. Teórica'] = df_principal['Qtde. Teórica'].astype(int)
df_principal

In [ ]:
df_principal = df_principal.rename(columns={'Qtde. Teórica':'qtd_teorica'}).copy()
df_principal

In [ ]:
df_principal['Resultado'] = df_principal['Variação_rs'].apply(lambda x: 'Subiu' if x > 0 else ('Desceu' if x < 0 else 'Estável'))
df_principal

In [ ]:
df_principal = df_principal.merge(df_tickers, left_on='Ativo', right_on='Ticker', how='left')
df_principal = df_principal.drop(columns=['Ticker'])
df_principal

In [ ]:
df_principal = df_principal.merge(df_chatgpt, left_on='Nome', right_on='Nomes das Empresas', how='left')
df_principal = df_principal.drop(columns=['Nomes das Empresas'])
df_principal

In [ ]:
df_principal['cat_idade'] = df_principal['Idade'].apply(lambda x: 'Mais de 100' if x > 100 else ('Menos de 50' if x < 50 else 'Entre 50 e 100'))
df_principal

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

maior = df_principal['Variação_rs'].max()
menor = df_principal['Variação_rs'].min()
media_total = df_principal['Variação_rs'].mean()

media_subiu = df_principal[df_principal['Resultado'] == 'Subiu']['Variação_rs'].mean()
media_desceu = df_principal[df_principal['Resultado'] == 'Desceu']['Variação_rs'].mean()

print(f"Maior\tR$ {maior:,.2f}")
print(f"Menor\tR$ {menor:,.2f}")
print(f"Média\tR$ {media_total:,.2f}")
print(f"Média de quem subiu\tR$ {media_subiu:,.2f}")
print(f"Média de quem desceu\tR$ {media_desceu:,.2f}")

In [ ]:
df_principal_subiu = df_principal[df_principal['Resultado'] == 'Subiu']
df_principal_subiu

In [ ]:
df_analise_segmento = df_principal_subiu.groupby('Categoria')['Variação_rs'].sum().reset_index()
df_analise_segmento

In [ ]:
df_analise_saldo = df_principal.groupby('Resultado')['Variação_rs'].sum().reset_index()
df_analise_saldo

In [ ]:
fig = px.bar(df_analise_saldo, x='Resultado', y='Variação_rs', text='Variação_rs', title='Variação Reais por Resultado')
fig.show()